<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル3

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [ ]:
!pip3 install optuna

  Using cached optuna-3.2.0-py3-none-any.whl (390 kB)
  Using cached alembic-1.11.1-py3-none-any.whl (224 kB)
  Using cached cmaes-0.9.1-py3-none-any.whl (21 kB)
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
  Using cached Mako-1.2.4-py3-none-any.whl (78 kB)


# 分散並列最適化


大規模な機械学習モデルを学習する場合は、1回のトライアルにかかる時間が長くなります。  
そのような時はハイパーパラメータの最適化を並列化したい場合があります。

Optunaを用いると、分散並列ハイパーパラメータ最適化を簡単に行う事ができます。   
Optunaの一般の分散並列最適化はRDBを用いるため、GoogleColabでは簡易的にできません。  
実行可能な環境で以下手順を実施してください。  
引用ノートブック：https://github.com/pfnet-research/optuna-hands-on/blob/master/ja/02_Optuna_Advanced_Tutorial_ja.ipynb

## Optunaの分散並列最適化の仕組み

全ての最適化履歴は、MySQLやPostgreSQLなどのRDB等の共有のストレージに保存されます。  
そして、実際に最適化を実行する複数のワーカーは、それぞれが独立にこのストレージにアクセスします。
そのため任意のワーカーを追加可能で中断ができます。またワーカー間の競合は、Optunaが管理をしてくれるのでユーザは気を遣う必要がありません。


In [ ]:
import optuna

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST

import sys

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # あればGPUを使う

# データを読み込む (今回はデータが少ないので、全てメモリ上に入れる)
# 訓練用データセット
train = MNIST(root='./data', download=True, train=True)
train_X = train.data.to(device).to(torch.float32) # 60000x28x28
train_y = train.targets.to(device) # 60000

# 評価用データセット
validation = MNIST(root='./data', download=True, train=False)
validation_X = validation.data.to(device).to(torch.float32) # 10000x28x28
validation_y = validation.targets.to(device) # 10000

<ipython-input-11-94271d1de426>:1: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  strage = optuna.storages.JournalStorage(optuna.storages.JournalFileStorage("./mnist_study.optuna"))
